In [1]:
import pandas as pd
import numpy as np

In [2]:
feattable = pd.read_csv("193398_feattable.csv")

In [3]:
train_matrix = pd.read_csv("193398_train.csv")

In [4]:
train_matrix = train_matrix[train_matrix['ocag_id_all_officerage_max']!=0]

In [5]:
import sys
sys.path.append('/Users/damoncrockett/differ')
import differ

In [6]:
tmp = train_matrix.drop(['as_of_date','outcome'],axis=1).apply(sum,axis=0) > 0
zcols = tmp.index[tmp==False]
train_matrix = train_matrix.drop(zcols,axis=1)

tmp = train_matrix.drop(['as_of_date','outcome'],axis=1)
ocols = [col for col in tmp.columns if len(tmp[col].value_counts())==1]
train_matrix = train_matrix.drop(ocols,axis=1)

In [7]:
dcols = train_matrix.drop(['as_of_date','outcome'],axis=1).columns
train_matrix[dcols] = train_matrix[dcols].applymap(float)

In [8]:
cols = train_matrix.columns
feattable = feattable[feattable.feature.isin(cols)]
feattable.reset_index(drop=True,inplace=True)

In [9]:
feattable_parsed = feattable

In [10]:
x = len(feattable_parsed)

In [11]:
feattable_parsed.sort_values(by='feature_importance',ascending=False,inplace=True)
top_feats = feattable_parsed.feature.iloc[:x]

In [12]:
cfeats = []
ifeats = []
featdists = []
subdists = []

nbins = 3 # for continuous features

for cfeat in top_feats:
    print cfeat
    for i in feattable_parsed.index:
        cfeats.append(cfeat)
        ifeat = feattable_parsed.feature.loc[i]
        ifeats.append(ifeat)
        
        if feattable_parsed.record_type.loc[i]=='ocnd':
            outputs = differ.KL_categorical(train_matrix,ifeat,cfeat)
        else:
            outputs = differ.KL_continuous(train_matrix,ifeat,cfeat,nbins)
        
        featdists.append(outputs[0])
        subdists.append(outputs[1])
        
df = pd.DataFrame({
        "candidate_feature":cfeats,
        "interacting_feature":ifeats,
        "feature_distance":featdists,
        "subset_distances":subdists
    })

dispatch_id_p1m_dispatchinitiatiationtype_ci_sum
ocag_id_all_officerage_max
dispatch_id_p1y_dispatchtype_assault_sum
dispatch_id_p1y_dispatchtype_assist_sum
dispatch_id_p1y_dispatchinitiatiationtype_ci_sum
dispatch_id_p1m_dispatchtype_disorder_sum
dispatch_id_p1y_dispatchtype_shooting_sum
dispatch_id_p1y_dispatchtype_domestic_disturb_sum
dispatch_id_p1m_dispatchtype_domestic_disturb_sum
dispatch_id_p1m_dispatchtype_theft_sum
dispatch_id_p1y_dispatchtype_intoxicated_avg
dispatch_id_p1w_dispatchinitiatiationtype_ci_sum
dispatch_id_p1m_dispatchtype_assist_sum
dispatch_id_p1m_dispatchtype_traffic_sum
dispatch_id_p1y_dispatchtype_intoxicated_sum
arrests_id_p5y_suspectsarrestedofrace_unknown_avg
shifts_id_p5y_hourspershift_sum
ts_id_p1y_trafficstopsbystoptype_equipment_violation_sum
dispatch_id_p5y_dispatchtype_assault_avg
dispatch_id_p1y_dispatchtype_drug_sum
dispatch_id_p1m_dispatchtype_domestic_disturb_avg
ts_id_p1y_trafficstopsbyrace_black_sum
dispatch_id_p1y_dispatchtype_suspicious_sum


In [13]:
df.to_csv("/Users/damoncrockett/Desktop/tmpwintour/193398-differ-3bin.csv",index=False)